![](../img/330-banner.png)

Lecture 8: Hyperparameter Optimization and Optimization Bias
------------

UBC 2022-23 W2

Instructor: Amir Abdi
 - Office Hours: Mondays 5-6 (or 5-7 if student turn-out was high)

<br><br><br>


## Please share your Feedback

<img src="./img_aa/qr-code-feedback-Amir.png" height="300" width="300"> 

https://forms.gle/NLKCe3BrLC6tLpkE8

It's 100% **anonymous**

<br><br><br><br>

<br><br><br><br><br><br>
There is a lot of contents we want to cover in each session; but, time is limited.

I jump over some cells in the Notebook; and **I trust that you will study them at home**.


<br><br>
**Reminder:** 
- Contents in the notebooks are for you to study; we won't be going through every cell in the 80-minute class. 
- Goal of the instructor sessions is to **learn Machine Learning concepts**.
- Goal of HW and Tutorials is to **learn to Apply those concepts (i.e., to code)**
<br><br><br><br><br><br>

## Announcements, and LO

## Imports

In [ ]:
import os
import sys

sys.path.append("../code/.")

import matplotlib.pyplot as plt
import mglearn
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
pd.set_option("display.max_colwidth", 200)

<br><br>

## Announcements

- HW3 was due last night. 
- HW4 is released.
- Midterm is coming up: **Feb 15 Wednesday**

## Learning outcomes 

From this lecture, you will be able to 

- explain the need for **hyperparameter optimization**
- carry out hyperparameter optimization using `sklearn`'s `GridSearchCV` and `RandomizedSearchCV` 
- explain different hyperparameters of `GridSearchCV`
- explain the importance of selecting a good range for the values. 
- explain optimization bias
- identify and reason when to trust and not trust reported accuracies 

<br><br><br><br>

# Hyperparameter optimization motivation 

## Motivation

- Remember that the fundamental goal of supervised machine learning is to generalize beyond what we see in the training examples. 
- We have been using data splitting and cross-validation to provide a framework to approximate generalization error.  
- With this framework, we can improve the model's generalization performance by tuning model hyperparameters using cross-validation on the training set. 

## Hyperparameters: the problem

- In order to improve the generalization performance, finding the best values for the important hyperparameters of a model is necessary. 
- Picking good hyperparameters is important because if we don't do it, we might end up with an underfit or overfit model. 

<br><br><br><br><br><br>
**Every decision you make in Machine Learning is a Hyper-Parameter (HParam):**
- How to clean data
- Which features to include 
- How to scale data
- Which ML model to choose
- Which parameters to decide for the model 
  - max-depth in tree, 
  - C value in SVM, 
  - gamma value in RBF, 
  - alpha value in Ridge Regression, 
  - C value in Logistic Regression
- ...
<br><br><br><br><br><br>

## Some ways to pick hyperparameters:
- Expert knowledge 
- Heuristics
- Search hyper-parameter space
- Data-driven or automated optimization

### Manual hyperparameter optimization

- Advantage: we may have some intuition about what might work.
  - E.g. if I'm massively overfitting, try decreasing `max_depth` or `C`.
- Disadvantages
    - it takes a lot of work
    - not reproducible, **not scalable**
    - in very complicated cases, our intuition might be worse than a data-driven approach

### Automated hyperparameter optimization 

- Formulate the hyperparamter optimization as a one big **search problem**. 
- Often we have many hyperparameters of different types: Categorical, integer, and continuous.
- Often, the search space is quite big and systematic search for optimal values is infeasible. 

In homework assignments, we have been carrying out hyperparameter search by exhaustively trying different possible combinations of the hyperparameters of interest. 

In [ ]:
mglearn.plots.plot_grid_search_overview()

Let's look at an example of tuning `max_depth` of the `DecisionTreeClassifier` on the Spotify dataset. 

In [ ]:
spotify_df = pd.read_csv("../data/spotify.csv", index_col=0)
X_spotify = spotify_df.drop(columns=["target", "song_title", "artist"])
y_spotify = spotify_df["target"]
X_spotify.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_spotify, y_spotify, test_size=0.2, random_state=123
)

Try **DecisionTreeClassifier**

In [ ]:
best_score = 0

# --------- ATTENTION HERE ----------------
param_grid = {"max_depth": np.arange(1, 20, 2)}
# --------- ATTENTION HERE ----------------

results_dict = {"max_depth": [], "mean_cv_score": []}
for depth in param_grid["max_depth"]:  # for each combination of parameters, train an SVC
    dt = DecisionTreeClassifier(max_depth=depth)
    scores = cross_val_score(dt, X_train, y_train)  # perform cross-validation
    mean_score = np.mean(scores)  # compute mean cross-validation accuracy

    if (mean_score > best_score):  # if we got a better score, store the score and parameters
        best_score = mean_score
        best_params = {"max_depth": depth}

    results_dict["max_depth"].append(depth)
    results_dict["mean_cv_score"].append(mean_score)

In [ ]:
pd.DataFrame(results_dict).sort_values(by="mean_cv_score", ascending=False).head(10)

In [ ]:
print(best_params, ', score:', best_score)

Try **SVM RBF** and tuning `C` and `gamma` on the same dataset. 

In [ ]:
pipe_svm = make_pipeline(StandardScaler(), SVC())  # We need scaling for SVM RBF
pipe_svm.fit(X_train, y_train)

Let's try cross-validation with default hyperparameters of SVC. 

In [ ]:
scores = cross_validate(pipe_svm, X_train, y_train, return_train_score=True)
print('validation score:', pd.DataFrame(scores).mean()['test_score'])

Now let's try exhaustive hyperparameter search using for loops. 

This is what we have been doing for this (**pseudocode warning**):

```
for gamma in [0.01, 1, 10, 100]: # for some values of gamma
    for C in [0.01, 1, 10, 100]: # for some values of C
        for fold in folds:
            fit in training portion with the given C
            score on validation portion
        compute average score
        
pick hyperparameter values which yield with best average score
```

In [ ]:
best_score = 0

# --------- ATTENTION HERE ----------------
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "gamma": [0.001, 0.01, 0.1, 1, 10, 100],
}
# --------- ATTENTION HERE ----------------

results_dict = {"C": [], "gamma": [], "mean_cv_score": []}

for gamma in param_grid["gamma"]:
    for C in param_grid["C"]:  # for each combination of parameters, train an SVC

        pipe_svm = make_pipeline(StandardScaler(), SVC(gamma=gamma, C=C))
        scores = cross_val_score(pipe_svm, X_train, y_train, cv=5)  # perform cross-validation
        mean_score = np.mean(scores)  # compute mean cross-validation accuracy
        if (mean_score > best_score):  # if we got a better score, store the score and parameters
            best_score = mean_score
            best_parameters = {"C": C, "gamma": gamma}
            
        results_dict["C"].append(C)
        results_dict["gamma"].append(gamma)
        results_dict["mean_cv_score"].append(mean_score)

In [ ]:
print(best_parameters, ", best_score:", best_score)

In [ ]:
df = pd.DataFrame(results_dict)

In [ ]:
df.sort_values(by="mean_cv_score", ascending=False).head(20)

plot the mean cross-validation scores

In [ ]:
scores = np.array(df.mean_cv_score).reshape(6, 6)

mglearn.tools.heatmap(
    scores,
    xlabel="C",
    xticklabels=param_grid["C"],
    ylabel="gamma",
    yticklabels=param_grid["gamma"],
    cmap="viridis",
)


- We have 6 possible values for `C` and 6 possible values for `gamma`. 
- In 5-fold cross-validation, for each combination of parameter values, five accuracies are computed.
- So to evaluate the accuracy of the SVM using 6 values of `C` and 6 values of `gamma` using five-fold cross-validation, **we need to train 36 * 5 = 180 models**! 

Once we have optimized hyperparameters, we retrain a model on the full training set with these optimized hyperparameters. 

In [ ]:
pipe_svm = make_pipeline(StandardScaler(), SVC(**best_parameters))
pipe_svm.fit(
    X_train, y_train
)  # Retrain a model with optimized hyperparameters on the combined training and validation set

And finally evaluate the performance of this model on the test set. 

In [ ]:
pipe_svm.score(X_test, y_test)  # Final evaluation on the test data

This process is so common that there are some standard methods in `scikit-learn` where we can carry out all of this in a more compact way. 

In [ ]:
mglearn.plots.plot_grid_search_overview()

In this lecture we are going to talk about two such most commonly used automated optimizations methods from `scikit-learn`. 

- Exhaustive grid search: [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- Randomized search: [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

The "CV" stands for cross-validation; these methods have built-in cross-validation.

<br><br><br><br>

# Exhaustive grid search: [`sklearn.model_selection.GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

- For `GridSearchCV` we need
    - an instantiated model or a pipeline
    - a parameter grid: A user specifies a set of values for each hyperparameter. 
    - other optional arguments 

The method considers product of the sets and evaluates each combination one by one.    

In [ ]:
from sklearn.model_selection import GridSearchCV

pipe_svm = make_pipeline(StandardScaler(), SVC())

# ---------- ATTENTION TO __ -----------
param_grid = {
    "svc__gamma": [0.001, 0.01, 0.1, 1.0, 10, 100],
    "svc__C": [0.001, 0.01, 0.1, 1.0, 10, 100],
}
# --------------------------------------


# ---------- NEW CODE -----------
grid_search = GridSearchCV(
    pipe_svm, param_grid, cv=5, n_jobs=-1, return_train_score=True
)
# ------------------------------


`n_jobs=-1` PARAMETER --> be patient :-)

In [ ]:
from sklearn import set_config
set_config(display="diagram")

grid_search

The `GridSearchCV` object above behaves like a classifier. We can call `fit`, `predict` or `score` on it. 

In [ ]:
# -------- NEW CODE ---------------
grid_search.fit(X_train, y_train) # all the work is done here
# -------- NEW CODE ---------------

Fitting the `GridSearchCV` object 
- Searches for the best hyperparameter values
- You can access the best score and the best hyperparameters using `best_score_` and `best_params_` attributes, respectively. 

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

- It is often helpful to visualize results of all cross-validation experiments. 
- You can access this information using `cv_results_` attribute of a fitted `GridSearchCV` object.  

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)
results.T

Let's only sort the dataframe on `rank_test_score` and look at the **most relevant rows**. 

In [ ]:
pd.DataFrame(grid_search.cv_results_)[
    [
        "mean_test_score",
        "param_svc__gamma",
        "param_svc__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

<br><br><br><br><br><br><br><br>
- Other than searching for best hyperparameter values, `GridSearchCV` also fits a new model on the whole training set with the parameters that yielded the best results. 
- So we can conveniently call `score` on the test set with a fitted `GridSearchCV` object. 

In [ ]:
grid_search.score(X_test, y_test)

**Question for the class:** Why are `best_score_` and the score above different? 
<br><br><br><br><br><br><br>

In [ ]:
# Same code copied here to cover 3 concepts: n_jobs, __, and exponential values
param_grid = {
    "svc__gamma": [0.001, 0.01, 0.1, 1.0, 10, 100],
    "svc__C": [0.001, 0.01, 0.1, 1.0, 10, 100],
}
grid_search = GridSearchCV(
    pipe_svm, param_grid, cv=5, n_jobs=-1, return_train_score=True
)

## `n_jobs=-1`
- Note the `n_jobs=-1` above.
- Hyperparameter optimization can be done **_in parallel_** for each of the configurations (utilizing multi-cores of CPU)
- This is very useful when scaling up to large numbers of machines in the cloud.

## The `__` syntax 

- Above: we have a nesting of transformers.
- We can access the parameters of the "inner" objects by using __ to go "deeper":
- `svc__gamma`: the `gamma` of the `svc` of the pipeline
- `svc__C`: the `C` of the `svc` of the pipeline

## Values of the Grid

- Note the **exponential increments** for `C` and `gamma`. 
- This is quite common.
- There is no point trying $C=\{1,2,3\ldots,100\}$ because $C=1,2,3$ are too similar to each other.
- Often we're trying to find an order of magnitude, e.g. $C=\{0.01,0.1,1,10,100\}$. 
- We can also write that as $C=\{10^{-2},10^{-1},10^0,10^1,10^2\}$. 
- Or, in other words, $C$ values to try are $10^n$ for $n=-2,-1,0,1,2$ which is basically what we have above.

<br><br>

## Visualizing the parameter grid as a heatmap 

In [ ]:
def fit_and_display_heatmap(param_grid, pipe, X_train, y_train, reshape=(6, 6)):

    # Define grid and Fit
    grid_search = GridSearchCV(
        pipe, param_grid, cv=5, n_jobs=-1, return_train_score=True
    )
    grid_search.fit(X_train, y_train)
        
    results = pd.DataFrame(grid_search.cv_results_)
    scores = np.array(results.mean_test_score).reshape(reshape)

    # plot the mean cross-validation scores
    mglearn.tools.heatmap(
        scores,
        xlabel="gamma",
        xticklabels=param_grid["svc__gamma"],
        ylabel="C",
        yticklabels=param_grid["svc__C"],
        cmap="viridis",
    );

- Note that the range we pick for the parameters play an important role in hyperparameter optimization. 
- For example, consider the following grid and the corresponding results.  

In [ ]:
param_grid1 = {
    "svc__gamma": 10.0**np.arange(-3, 3, 1), 
    "svc__C": 10.0**np.arange(-3, 3, 1)
}
fit_and_display_heatmap(param_grid1, pipe_svm, X_train, y_train)

- Each point in the heat map corresponds to one run of cross-validation, with a particular setting
- Colour encodes cross-validation accuracy. 
    - Lighter colour means high accuracy
    - Darker colour means low accuracy
- SVC is quite sensitive to hyperparameter settings.
- Adjusting hyperparameters can change the accuracy from 0.51 to 0.74! 

---------------
**[study on your own]**

## Bad range for hyperparameters 

In [ ]:
np.logspace(1, 2, 6)

In [ ]:
np.linspace(1, 2, 6)

In [ ]:
param_grid2 = {"svc__gamma": np.round(np.logspace(1, 2, 6), 1), "svc__C": np.linspace(1, 2, 6)}
fit_and_display_heatmap(param_grid2, pipe_svm, X_train, y_train)

## Different range for hyperparameters yields better results! 

In [ ]:
np.logspace(-3, 2, 6)

In [ ]:
np.linspace(1, 2, 6)

In [ ]:
param_grid3 = {"svc__gamma": np.logspace(-3, 2, 6), "svc__C": np.linspace(1, 2, 6)}

fit_and_display_heatmap(param_grid3, pipe_svm, X_train, y_train)

It seems like we are getting even better cross-validation results with `C` = 2.0 and `gamma` = 0.1 

How about exploring different values of `C` close to 2.0? 

In [ ]:
param_grid4 = {"svc__gamma": np.logspace(-3, 2, 6), "svc__C": np.linspace(2, 3, 6)}

fit_and_display_heatmap(param_grid4, pipe_svm, X_train, y_train)

That's good! We are finding some more options for `C` where the accuracy is 0.75. 
The tricky part is we do not know in advance what range of hyperparameters might work the best for the given problem, model, and the dataset.    


`GridSearchCV` allows the param_grid to be a list of dictionaries. 

Each model has its own hyper-parameters:
- in `SVC` with **RBF** kernel: `C` and `gamma` are applicable 
- in `SVC` with **linear** kernel: `C` is applicable


**[end of study on your own]**

---------------

<br><br><br><br><br><br>


## Problems with exhaustive grid search 

- Required number of models to evaluate **grows exponentially** with the dimensionally of the configuration space. 
- Example: Suppose you have
    - 5 hyperparameters 
    - 10 different values for each hyperparameter
    - You'll be evaluating $10^5=100,000$ models! That is you'll be calling `cross_validate` 100,000 times!
- Exhaustive search becomes **infeasible (intractable)** fairly quickly. 

![grid-search](https://www.yourdatateacher.com/wp-content/uploads/2021/03/image-6.png)

But, there is a **another important problem with Grid Search**:

- Waste of compute in retrying the **same** Hyper-Param values multiple times
  - Example: in the following figure, values of C=0.001 and C=0.01 are not producing good models; but, we keep on retrying them and wasting compute.
- Adding **HParams that do not influence the performance** adversely impacts the **efficiency** (more wasted compute).

In [ ]:
param_grid1 = {
    "svc__gamma": 10.0**np.arange(-3, 1, 1), 
    "svc__C": 10.0**np.arange(-3, 3, 1)
}
fit_and_display_heatmap(param_grid1, pipe_svm, X_train, y_train, (6, 4))

<br><br><br><br>

# Randomized hyperparameter search

- Randomized hyperparameter optimization 
    - [`sklearn.model_selection.RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- Samples configurations at random until certain budget (e.g., time) is exhausted 

In [ ]:
param_grid = {
    "svc__gamma": [0.001, 0.01, 0.1, 1.0, 10, 100],
    "svc__C": np.linspace(2, 3, 6),
}

print("Grid size: %d" % (np.prod(list(map(len, param_grid.values())))))
param_grid

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# ---------- New Code ----------------
random_search = RandomizedSearchCV(
    pipe_svm, param_distributions=param_grid, n_jobs=-1, n_iter=20, cv=5, random_state=42
)
random_search.fit(X_train, y_train);
# ---------- New Code ----------------

Notice the new parameter: **n_iter**

`n_iter`

- Note the `n_iter`, we didn't need this for `GridSearchCV`.
- Larger `n_iter` will take longer but it'll do more searching.
  - Remember you still need to multiply by number of folds!
- I have set the `random_state` for **reproducibility of the HParam optimization experiment**.

In [ ]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_score",
        "param_svc__gamma",
        "param_svc__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

<br><br><br><br><br><br>
**Reminder**:

**An important problem with Grid Search**: Waste of compute in retrying the **same** Hyper-Param values multiple times

**Question**: Did we solve the above problem in our `RandomizedSearchCV` solution?  
Answer: ????

<br><br>

## Probability distributions for Random Search (instead of a predefined list!)

Another thing we can do is give probability distributions to draw from:

In [ ]:
from scipy.stats import expon, lognorm, loguniform, randint, uniform, norm

In [ ]:
np.random.seed(123)

# -------- New Code -----------
y = uniform.rvs(0, 5, 10000) # --> sampling 10,000 random numbers, from a uniform ditribution, between 0 and 5
# -------- New Code -----------

bin = np.arange(-3,8,0.1)  
plt.hist(y, bins=bin, edgecolor='blue') 
plt.show()

In [ ]:
# -------- New Code -----------
y = norm.rvs(3, 2, 10000) # --> sampling 10,000 random numbers, from a normal distribution, with mean=3 and std=2
# -------- New Code -----------

#creating bin
bin = np.arange(-4,10,0.1)  
plt.hist(y, bins=bin, edgecolor='blue') 
plt.show()

In [ ]:
# -------- New Code -----------
y = expon.rvs(5, 2, 10000) # --> sampling 10,000 random numbers, from a exponential distribution, at location=5 and scale=2
# -------- New Code -----------

#creating bin
bin = np.arange(-1,10,0.1)  

plt.hist(y, bins=bin, edgecolor='blue') 
plt.show()

<br><br>

In [ ]:
y

In [ ]:
# -------- New Code -----------
y = loguniform.rvs(0.001, 100, scale=1000, size=10000) # --> sampling 10,000 random numbers, from a loguniform distribution
# -------- New Code -----------

#creating bin
bin = np.arange(0.001,100,1)  

plt.hist(y, bins=bin, edgecolor='blue') 
plt.show()

In [ ]:
# Passing uniform and loguniform as parameters to the RandomSearch to sample from
param_dist = {
    "svc__C": loguniform(0.001, 100),
    "svc__gamma": loguniform(0.001, 100),
}
random_search = RandomizedSearchCV(
    pipe_svm, param_dist, n_iter=100, verbose=1, n_jobs=-1, random_state=123
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_score_

In [ ]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_score",
        "param_svc__gamma",
        "param_svc__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

- This is a bit fancy. What's nice is that you can have it concentrate more on certain values by setting the distribution. 


### Advantages of `RandomizedSearchCV`

- Faster compared to `GridSearchCV`.
- Adding parameters that do not influence the performance does not harm efficiency.
- It is recommended to use `RandomizedSearchCV` rather than `GridSearchCV`.    

Check the 2012 paper from **Yoshua Bengio*** on [Random Search for Hyper-Parameter Optimization](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf)

-----------
<img src="https://upload.wikimedia.org/wikipedia/commons/1/1e/Yoshua_Bengio_2019_cropped.jpg" width="300">

Yoshua Bengio  
1964-Now  
- Université de Montréal and scientific director of the Montreal Institute for Learning Algorithms (**MILA**).
- Received the 2018 ACM A.M. **Turing Award**

-----------

### Advantages of `RandomizedSearchCV`

![](../img/randomsearch_bergstra.png)
    
Source: [Bergstra and Bengio, Random Search for Hyper-Parameter Optimization, JMLR 2012](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf).

- The yellow on the left shows how your scores are going to change when you vary the unimportant hyperparameter.
- The green on the top shows how your scores are  going to change when you vary the important hyperparameter.
- You don't know in advance which hyperparameters are important for your problem.
- In the left figure, 6 of the 9 searches are useless because they are only varying the unimportant parameter.
- In the right figure, all 9 searches are useful.

# Searching for optimal parameters with successive halving¶

Successive halving is an iterative selection process which **initially** evluates all candidates with **small amount of resources** and then **iteratively adds more and more resources.**

<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_successive_halving_iterations_001.png" width="700">

Learn more here: https://scikit-learn.org/stable/modules/grid_search.html#successive-halving-user-guide

And be aware of different variations for GridSearch and RandomSearch:
- [successive halving with grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html)
- [random search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html).

# [Optional / Bonus] Model-based HParam Optimization
- Both `GridSearchCV` and `RandomizedSearchCV` do **each trial independently**
  - Outcome of one trial **does not impact HParam decisions** of the next trials.
- What if you could learn from your experience, e.g. learn that `max_depth=3` is bad?
  - That could save time because you wouldn't try combinations involving `max_depth=3` in the future.
- We can do this with `scikit-optimize`, which is a completely different package from `scikit-learn`
- It uses a technique called "model-based optimization" and we'll specifically use "**Bayesian optimization**".
  - In short, it uses machine learning to predict what hyperparameters will be good.
  - **Machine learning on machine learning!**


<br><br><br><br><br>
Model-based HParam optimization is like **two layers** of Machine Learning
<br><br>

<img src="https://github.com/fmfn/BayesianOptimization/raw/master/examples/bo_example.png" width="1000">

img source: https://github.com/fmfn/BayesianOptimization
<br><br><br><br><br>



- This is an active research area and there are sophisticated packages for this.

Here are some examples 
- [hyperopt-sklearn](https://github.com/hyperopt/hyperopt-sklearn)
- [auto-sklearn](https://github.com/automl/auto-sklearn)
- [SigOptSearchCV](https://sigopt.com/docs/overview/scikit_learn)
- [TPOT](https://github.com/rhiever/tpot)
- [hyperopt](https://github.com/hyperopt/hyperopt)
- [hyperband](https://github.com/zygmuntz/hyperband)
- [SMAC](http://www.cs.ubc.ca/labs/beta/Projects/SMAC/)
- [MOE](https://github.com/Yelp/MOE)
- [pybo](https://github.com/mwhoffman/pybo)
- [spearmint](https://github.com/HIPS/Spearmint)
- [BayesOpt](https://github.com/rmcantin/bayesopt)

## ❓❓ Questions for you

## (iClicker) Exercise 8.1 

**iClicker cloud join link: https://join.iclicker.com/EMMJ**

**Select all of the following statements which are TRUE.**

- (A) If you get best results at the **edges of your parameter grid**, it might be a good idea to adjust the range of values in your parameter grid.   
- (B) Grid search is **guaranteed** to find best hyperparameters values. 
- (C) In general, it is possible to get **different** hyperparameters in **different runs** of `RandomizedSearchCV`.

Answer = ????

## (iClicker) Exercise 8.2 

Suppose you have 10 hyperparameters, each with 4 possible values.   
If you run `GridSearchCV` with this parameter grid,  
how many cross-validation experiments will be carried out? 

Answer: ????

## (iClicker) Exercise 8.3

Suppose you have 10 hyperparameters, each with 4 possible values.   
If you run `RandomizedSearchCV` with this parameter grid with `n_iter=20`,  
how many cross-validation experiments will be carried out? 

Answer: ?????

<br><br><br><br>

# Optimization bias/Overfitting of the validation set

## Overfitting of the validation error 

- **Why do we need to evaluate the model on the test set in the end?**
  - Why do we have the test Set?
  - Why isn't Validation Set enough?
- Why not just use cross-validation on the whole dataset? 
- While carrying out hyperparameter optimization, we usually try over many possibilities.  

<br><br><br><br>
If your **validation set is hit too many times**, we suffer from **optimization bias** or **overfitting the validation set**. 
<br><br><br><br>

## Optimization bias of parameter learning (overfitting on train set)
- Overfitting of the training error
- An example: 
    - During training, we could search over tons of different decision trees.
    - So we can get "lucky" and find a tree with low training error by chance.

    

### Optimization bias of hyper-parameter learning (overfitting on validation set)

- Overfitting of the validation error
- An example: 
    - Here, we might optimize the validation error over 1000 values of `max_depth`.
    - One of the 1000 trees might have low validation error by chance.



-------------
**[Read at home; it's a fun experiment]**

### (Optional) Example 1: Optimization bias

Consider a multiple-choice (a,b,c,d) "test" with **10 questions**:
- If you choose answers randomly, expected grade is 25% (no bias).
- If you retry the same test twice, randomly, and pick the best, expected grade is 33%.
    - Optimization bias of ~8%.
- If you take the best among 10 random tests, expected grade is ~47%.
- If you take the best among 100, expected grade is ~62%.
- If you take the best among 1000, expected grade is ~73%.
- If you take the best among 10000, expected grade is ~82%.
    - You have so many "chances" that you expect to do well.
    
**But on new questions the "random choice" accuracy is still 25%.**

In [ ]:
# (Optional) Code attribution: Rodolfo Lourenzutti
number_tests = [1, 2, 10, 100, 1000, 10000]
for ntests in number_tests:
    y = np.zeros(10000)
    for i in range(10000):
        y[i] = np.max(np.random.binomial(10.0, 0.25, ntests))
    print(
        "The expected grade among the best of %d tests is : %0.2f"
        % (ntests, np.mean(y) / 10.0)
    )

### (Optional) Example 2: Optimization bias 

- If we instead used a **100-question** test then:
    - Expected grade from best over 1 randomly-filled test is 25%.
    - Expected grade from best over 2 randomly-filled test is ~27%.
    - Expected grade from best over 10 randomly-filled test is ~32%.
    - Expected grade from best over 100 randomly-filled test is ~36%.
    - Expected grade from best over 1000 randomly-filled test is ~40%.
    - Expected grade from best over 10000 randomly-filled test is ~43%.

- The optimization bias **grows with the number of things we try**.
    - “Complexity” of the set of models we search over.
- But, optimization bias **shrinks quickly with the number of examples**.
    - But it’s still non-zero and growing if you over-use your validation set!    

In [ ]:
# (Optional) Code attribution: Rodolfo Lourenzutti
number_tests = [1, 2, 10, 100, 1000, 10000]
for ntests in number_tests:
    y = np.zeros(10000)
    for i in range(10000):
        y[i] = np.max(np.random.binomial(100.0, 0.25, ntests))
    print(
        "The expected grade among the best of %d tests is : %0.2f"
        % (ntests, np.mean(y) / 100.0)
    )


**[End of Read at home; it's a fun experiment]**

------------

## Optimization bias on the Spotify dataset

In [ ]:
X_train_tiny, X_test_big, y_train_tiny, y_test_big = train_test_split(
    X_spotify, y_spotify, test_size=0.99, random_state=42
)

In [ ]:
X_train_tiny.shape

In [ ]:
X_train_tiny.head()

In [ ]:
pipe = make_pipeline(StandardScaler(), SVC())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# ----------- Attention Here -----------
param_grid = {
    "svc__gamma": 10.0 ** np.arange(-20, 10),
    "svc__C": 10.0 ** np.arange(-20, 10),
}
# ----------------------------------------
print("Grid size: %d" % (np.prod(list(map(len, param_grid.values())))))
param_grid

In [ ]:
# ----------- Attention Here: n_iter=900 -----------
random_search = RandomizedSearchCV(
    pipe, param_distributions=param_grid, n_jobs=-1, n_iter=900, cv=5, random_state=123
)
random_search.fit(X_train_tiny, y_train_tiny);
# ----------------------------------------

In [ ]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_test_score",
        "param_svc__gamma",
        "param_svc__C",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

Given the results: one might claim that we found a model that performs with 0.8 accuracy on our dataset. 

- Do we really believe that 0.80 is a good estimate of our test data?
- Do we really believe that `gamma`=0.0 and C=1_000_000_000 are the best hyperparameters? 

- Let's find out the test score with this best model. 

In [ ]:
random_search.score(X_test, y_test)

**`0.61` is much smaller than `0.8`; we have overfitted on the validation set!**

- The results above are overly optimistic. 
    - because our training data is very small and so our validation splits in cross validation would be small. 
    - because of the small dataset and the fact that **we hit the small validation set 900 times** and it's possible that we got lucky on the validation set! 

- As we suspected, the best cross-validation score is not a good estimate of our test data; it is overly optimistic. 
- We can trust this test score because the test set is of good size.

In [ ]:
X_test_big.shape

### Overfitting of the validation data

The following plot demonstrates what happens during overfitting of the validation data.  
<center>
<img src='../img/optimization-bias.png' width="600">
</center>


[Source](https://amueller.github.io/COMS4995-s20/slides/aml-03-supervised-learning/#20)

- Thus, not only can we not trust the cv scores, we also cannot trust cv's ability to choose of the best hyperparameters. 


### Why do we need a test set? 
- The frustrating part is that if **our dataset is small** then our **test set is also small**. 
- But we don't have a lot of better alternatives, unfortunately, if we have a small dataset. 

<br><br><br><br>
**The larger your data, less chance of all overfitting.**  
**The larger your data, more reliable are your reports (similar to statistical studies).**  
<br><br><br><br>

### When test score is much lower than CV score
- What to do if your test score is much lower than your cross-validation score:
    - Try simpler models and use the test set a couple of times; it's not the end of the world.
    - Communicate this clearly when you report the results. 

### Large datasets solve many of these problems
- With infinite amounts of training data, overfitting would not be a problem and you could have your test score = your train score.
    - Overfitting happens because you only see a bit of data and you learn patterns that are overly specific to your sample.
    - If you saw "all" the data, then the notion of "overly specific" would not apply.
- So, more data will make your test score better and robust. 

## ❓❓ Questions for you

Attribution: From Mark Schmidt's notes

### Would you trust the model?  

- You have a dataset and you give me half of it. I train a model using all the data you have given me and I tell you that the model accuracy is 0.99.   
**Would it classify the rest of the data with similar accuracy?** 

1. Probably 
2. Probably not 

### Would you trust the model?  

- You have a dataset and you give me half of it. I build a model using 80% of the data given to me and report the accuracy of 0.95 on the remaining 20% of the data.    
**Would it classify the rest of the data with similar accuracy?**

1. Probably 
2. Probably not 

### Would you trust the model?  

- You have a dataset and you give me 1/10th of it. The dataset given to me is rather small and so I split it into 96% train and 4% validation split. I carry out hyperparameter optimization using a single 4% validation split and report validation accuracy of 0.97.    
**Would it classify the rest of the data with similar accuracy?**

1. Probably 
2. Probably not 

## Final comments and summary

#### Automated hyperparameter optimization

- Advantages 
    - reduce human effort
    - less prone to error and improve reproducibility
    - data-driven approaches may be effective
- Disadvantages
    - may be hard to incorporate intuition
    - be careful about overfitting on the validation set
    - might require a lot of compute for a thorough HParam optimization if too many HParams

Often, especially on typical datasets, we get back `scikit-learn`'s default hyperparameter values. This means that the defaults are well chosen by `scikit-learn` developers!

- The problem of finding the best values for the important hyperparameters is tricky because 
    - You may have a lot of them (e.g. deep learning). 
    - You may have multiple hyperparameters which may interact with each other in unexpected ways.    
- The best settings depend on the specific data/problem.

## Optional readings and resources

- [Preventing "overfitting" of cross-validation data](http://www.robotics.stanford.edu/~ang/papers/cv-final.pdf) by Andrew Ng